Consulta a un libro con retrieval augmented generation

CARGA DE TEXTO

In [4]:
import pymupdf
import pymupdf as fitz

# Ruta al archivo PDF
path = "C:\\Users\\ELI\\Downloads\\irbookonlinereading.pdf"
doc = pymupdf.open(path)
data=[]
for i, page in enumerate(doc):
    text = page.get_text().encode('utf-8')
    if text.strip():
        data.append({
            "page": i + 1,
            "raw": str(text).strip()
        })
# Convertir a DataFrame
import pandas as pd
df = pd.DataFrame(data)
df

,page,raw
0,1,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...
1,2,b'Online edition (c)\n2009 Cambridge UP\n'
2,3,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...
3,4,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...
4,5,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...
...,...,...
576,577,b'Online edition (c)\n2009 Cambridge UP\n540\n...
577,578,b'Online edition (c)\n2009 Cambridge UP\nIndex...
578,579,b'Online edition (c)\n2009 Cambridge UP\n542\n...
579,580,b'Online edition (c)\n2009 Cambridge UP\nIndex...


TABLA DE CONTENIDOS

In [5]:
# Verificar si el PDF tiene tabla de contenidos (TOC)
toc = doc.get_toc()

if toc:
    print(f"¡El PDF tiene {len(toc)} elementos en la tabla de contenidos!")
    print("\nPrimeras 10 entradas:")
    for i, (level, title, page) in enumerate(toc[:100]):
        indent = "  " * (level - 1)
        print(f"{indent}Nivel {level}: {title} (Página {page})")
else:
    print("El PDF no tiene tabla de contenidos estructurada")

¡El PDF tiene 256 elementos en la tabla de contenidos!

Primeras 10 entradas:
Nivel 1: List of Tables (Página 15)
Nivel 1: List of Figures (Página 19)
Nivel 1: Table of Notation (Página 27)
Nivel 1: Preface (Página 31)
Nivel 1: Boolean retrieval (Página 38)
  Nivel 2: An example information retrieval problem (Página 40)
  Nivel 2: A first take at building an inverted index (Página 43)
  Nivel 2: Processing Boolean queries (Página 47)
  Nivel 2: The extended Boolean model versus ranked retrieval (Página 51)
  Nivel 2: References and further reading (Página 54)
Nivel 1: The term vocabulary and postings lists (Página 56)
  Nivel 2: Document delineation and character sequence decoding (Página 56)
    Nivel 3: Obtaining the character sequence in a document (Página 56)
    Nivel 3: Choosing a document unit (Página 57)
  Nivel 2: Determining the vocabulary of terms (Página 59)
    Nivel 3: Tokenization (Página 59)
    Nivel 3: Dropping common terms: stop words (Página 64)
    Nivel 3: Normali

PREPROCESAR EL TEXTO

In [6]:
#preprocesar el texto
import re
#normalizar texto
def normalize_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'\s+', ' ', text)  # Eliminar espacios en blanco extra
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar caracteres especiales
    return text
# Aplicar normalización al DataFrame
df['normalized'] = df['raw'].apply(normalize_text)
# Verificar el resultado
df

,page,raw,normalized
0,1,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...,bonline edition cn2009 cambridge upnannintrodu...
1,2,b'Online edition (c)\n2009 Cambridge UP\n',bonline edition cn2009 cambridge upn
2,3,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...,bonline edition cn2009 cambridge upnannintrodu...
3,4,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...,bonline edition cn2009 cambridge upndraftndo n...
4,5,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...,bonline edition cn2009 cambridge upndraft xc2x...
...,...,...,...
576,577,b'Online edition (c)\n2009 Cambridge UP\n540\n...,bonline edition cn2009 cambridge upn540nindexn...
577,578,b'Online edition (c)\n2009 Cambridge UP\nIndex...,bonline edition cn2009 cambridge upnindexn541n...
578,579,b'Online edition (c)\n2009 Cambridge UP\n542\n...,bonline edition cn2009 cambridge upn542nindexn...
579,580,b'Online edition (c)\n2009 Cambridge UP\nIndex...,bonline edition cn2009 cambridge upnindexn543n...


In [ ]:
#eliminar stopwords
from nltk.corpus import stopwords
# Definir stopwords en english
stop_words = set(stopwords.words('english'))
# Función para eliminar stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)
# Aplicar eliminación de stopwords al DataFrame
df['filtered'] = df['normalized'].apply(remove_stopwords)
# Verificar el resultado
df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ELI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,page,raw,normalized,filtered
0,1,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...,bonline edition cn2009 cambridge upnannintrodu...,bonline edition cn2009 cambridge upnannintrodu...
1,2,b'Online edition (c)\n2009 Cambridge UP\n',bonline edition cn2009 cambridge upn,bonline edition cn2009 cambridge upn
2,3,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...,bonline edition cn2009 cambridge upnannintrodu...,bonline edition cn2009 cambridge upnannintrodu...
3,4,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...,bonline edition cn2009 cambridge upndraftndo n...,bonline edition cn2009 cambridge upndraftndo d...
4,5,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...,bonline edition cn2009 cambridge upndraft xc2x...,bonline edition cn2009 cambridge upndraft xc2x...
...,...,...,...,...
576,577,b'Online edition (c)\n2009 Cambridge UP\n540\n...,bonline edition cn2009 cambridge upn540nindexn...,bonline edition cn2009 cambridge upn540nindexn...
577,578,b'Online edition (c)\n2009 Cambridge UP\nIndex...,bonline edition cn2009 cambridge upnindexn541n...,bonline edition cn2009 cambridge upnindexn541n...
578,579,b'Online edition (c)\n2009 Cambridge UP\n542\n...,bonline edition cn2009 cambridge upn542nindexn...,bonline edition cn2009 cambridge upn542nindexn...
579,580,b'Online edition (c)\n2009 Cambridge UP\nIndex...,bonline edition cn2009 cambridge upnindexn543n...,bonline edition cn2009 cambridge upnindexn543n...


EMBEDDING

In [8]:
from sentence_transformers import SentenceTransformer
import numpy as np


In [15]:
model = SentenceTransformer('all-MiniLM-L6-v2')
#generar funcion embeddings para todo el texto
def generate_embeddings(text):
    return model.encode(text)
# Aplicar la función de embeddings al DataFrame
df['embeddings'] = df['filtered'].apply(generate_embeddings)


In [16]:
df

,page,raw,normalized,filtered,embeddings
0,1,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...,bonline edition cn2009 cambridge upnannintrodu...,bonline edition cn2009 cambridge upnannintrodu...,"[-0.11581327, -0.04094364, 0.023199067, -0.036..."
1,2,b'Online edition (c)\n2009 Cambridge UP\n',bonline edition cn2009 cambridge upn,bonline edition cn2009 cambridge upn,"[-0.1323803, -0.023347827, 0.06720841, -0.0928..."
2,3,b'Online edition (c)\n2009 Cambridge UP\nAn\nI...,bonline edition cn2009 cambridge upnannintrodu...,bonline edition cn2009 cambridge upnannintrodu...,"[0.00705546, -0.048072208, 0.007882582, -0.068..."
3,4,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...,bonline edition cn2009 cambridge upndraftndo n...,bonline edition cn2009 cambridge upndraftndo d...,"[-0.05006524, -0.0019235705, -0.016345194, -0...."
4,5,b'Online edition (c)\n2009 Cambridge UP\nDRAFT...,bonline edition cn2009 cambridge upndraft xc2x...,bonline edition cn2009 cambridge upndraft xc2x...,"[0.027868386, -0.087042965, -0.01009805, 0.016..."
...,...,...,...,...,...
576,577,b'Online edition (c)\n2009 Cambridge UP\n540\n...,bonline edition cn2009 cambridge upn540nindexn...,bonline edition cn2009 cambridge upn540nindexn...,"[-0.026038677, -0.12275319, -0.014319798, -0.0..."
577,578,b'Online edition (c)\n2009 Cambridge UP\nIndex...,bonline edition cn2009 cambridge upnindexn541n...,bonline edition cn2009 cambridge upnindexn541n...,"[0.02994777, -0.13199653, -0.008621623, -0.010..."
578,579,b'Online edition (c)\n2009 Cambridge UP\n542\n...,bonline edition cn2009 cambridge upn542nindexn...,bonline edition cn2009 cambridge upn542nindexn...,"[0.0034598007, -0.077709176, -0.0045174663, -0..."
579,580,b'Online edition (c)\n2009 Cambridge UP\nIndex...,bonline edition cn2009 cambridge upnindexn543n...,bonline edition cn2009 cambridge upnindexn543n...,"[-0.020311879, -0.13165753, 0.01688339, -0.020..."


In [17]:
#sacar los embeddings del df
embeddings = np.array(df['embeddings'].tolist())
embeddings

array([[-0.11581327, -0.04094364,  0.02319907, ..., -0.0260279 ,
         0.02207126, -0.04298127],
       [-0.1323803 , -0.02334783,  0.06720841, ...,  0.01590319,
        -0.02216803,  0.01160852],
       [ 0.00705546, -0.04807221,  0.00788258, ...,  0.0255634 ,
        -0.02932286,  0.00203923],
       ...,
       [ 0.0034598 , -0.07770918, -0.00451747, ..., -0.01676263,
        -0.03796598, -0.0576323 ],
       [-0.02031188, -0.13165753,  0.01688339, ..., -0.06474322,
        -0.04680839, -0.02762038],
       [ 0.00927761, -0.12823053,  0.01517444, ..., -0.02842616,
        -0.03841518, -0.03689068]], dtype=float32)

QUERY

In [31]:
#query
query = "stopwords"
query_embedding = model.encode(query)
query_embedding

array([-1.49560859e-03,  1.38110016e-02,  3.26872654e-02, -3.50735500e-03,
       -4.26756963e-03, -6.79441215e-03,  1.45399123e-01, -1.11046331e-02,
       -7.86744952e-02,  2.04429794e-02,  6.93996670e-03, -2.10056063e-02,
        4.66357656e-02, -4.85497192e-02, -7.59261772e-02,  8.11752751e-02,
        4.30341251e-02,  4.64086644e-02, -9.42633674e-02, -9.08575580e-03,
        6.45952672e-02,  1.34171650e-01,  2.02286150e-02,  2.76836660e-02,
       -1.09036537e-02,  5.80462441e-02, -9.04709026e-02, -6.54042652e-03,
        6.67781010e-02, -8.32427386e-03,  2.57875044e-02,  2.84907110e-02,
        6.08631112e-02,  2.46394724e-02,  2.32363176e-02, -3.44091505e-02,
       -1.05311200e-01,  5.39251119e-02,  2.46874914e-02, -1.99775379e-02,
       -7.40725473e-02, -1.03850476e-01, -1.05458960e-01, -4.92498577e-02,
        1.12935327e-01, -8.43069144e-03, -2.37405039e-02,  1.15957484e-02,
        5.11254892e-02,  2.55397037e-02, -7.89895132e-02, -5.44321947e-02,
       -2.77857948e-02,  

SIMILITUD COSENO

In [40]:
#similitud coseno de la consulta hecho embedding con los embeddings
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity([query_embedding], embeddings)[0]
#mostrar los resultados
similarity_scores


array([ 0.14673242,  0.08636067,  0.17792588,  0.12479579,  0.3501364 ,
        0.08636067,  0.36092517,  0.3500868 ,  0.23611969,  0.3248849 ,
        0.24582897,  0.22838886,  0.21565823,  0.08636067,  0.17770323,
        0.2217893 ,  0.14515658,  0.08636067,  0.3878405 ,  0.3268564 ,
        0.20410894,  0.21630645,  0.16421896,  0.09596944,  0.15224014,
        0.08636067,  0.16589503,  0.20008476,  0.2729969 ,  0.14498296,
        0.2922377 ,  0.29981863,  0.21870802,  0.2557671 ,  0.12749767,
        0.16471665,  0.20196778,  0.31844175,  0.27793705,  0.3152651 ,
        0.3540063 ,  0.34254867,  0.30332682,  0.29782933,  0.28053147,
        0.2698514 ,  0.23350316,  0.2839531 ,  0.31669274,  0.26685682,
        0.24922231,  0.32965595,  0.30776873,  0.3519211 ,  0.26064637,
        0.38121098,  0.29442626,  0.25781736,  0.33245167,  0.29098356,
        0.35760248,  0.3939588 ,  0.3240416 ,  0.5092842 ,  0.28471947,
        0.2545432 ,  0.35800108,  0.2520738 ,  0.41189817,  0.42

In [41]:
# Obtener los índices de las 5 entradas más similares
top_k_indices = np.argsort(similarity_scores)[-5:][::-1]
top_k_indices

array([ 63, 275, 468,  69,  85], dtype=int64)

In [ ]:
#imprimir los 5 resultados más similares de cuando ya esta preprocesado
print("Resultados más similares:")
for index in top_k_indices:
    print(f"Página {df.iloc[index]['page']}: {df.iloc[index]['filtered'][:200]}...")  
    print(f"Similitud: {similarity_scores[index]:.4f}\n")


Resultados más similares:
Página 64: bonline edition cn2009 cambridge upn22ndetermining vocabulary termsn27n222ndropping common terms stop wordsnsometimes extremely common words would appear ofnlittle value helping select documents match...
Similitud: 0.5093

Página 276: bonline edition cn2009 cambridge upn121nlanguage modelsn239nmodel m1nmodel m2nthen02nthen015nan01nan012nfrogn001nfrogn00002ntoadn001ntoadn00001nsaidn003nsaidn003nlikesn002nlikesn004nthatn004nthatn004n...
Similitud: 0.4266

Página 469: bonline edition cn2009 cambridge upn432n19nweb search basicsn194nthe search user experiencenit crucial understand users web search well isnagain signixefxacx81cant change traditional information retri...
Similitud: 0.4210

Página 70: bonline edition cn2009 cambridge upn22ndetermining vocabulary termsn33nlinguistic processing stemming lemmatization often done adnditional plugin component indexing process number suchncomponents exis...
Similitud: 0.4200

Página 86: bonline edition cn2009 ca

In [34]:
#unir los 5 resultados más similares
context = " ".join(df.iloc[index]['filtered'] for index in top_k_indices)
# Imprimir el contexto
print("Contexto de los resultados más similares:")
print(context)


Contexto de los resultados más similares:
bonline edition cn2009 cambridge upn22ndetermining vocabulary termsn27n222ndropping common terms stop wordsnsometimes extremely common words would appear ofnlittle value helping select documents matching user need excludednfrom vocabulary entirely words called stop words generalnstop wordsnstrategy determining stop list sort terms collection frequencyncollectionnfrequencynthe total number times term appears document collectionnand take frequent terms often handxefxacx81ltered senmantic content relative domain documents indexed asna stop list members discarded indexing annstop listnexample stop list shown figure 25 using stop list signixefxacx81cantlynreduces number postings system store presentnsome statistics chapter 5 see table 51 page 87 lot ofnthe time indexing stop words little harm keyword searches withnterms like donxe2x80x99t seem useful however true fornphrase searches phrase query xe2x80x9cpresident united statesxe2x80x9d conntains tw

INTEGRACIÓN CON IA

In [35]:
prompt = f"""Eres una aplicación de Retrieval Augmented Generation que siempre responde en español. Usa el siguiente contexto para responder la pregunta.
Si la respuesta no está en el contexto, di que no sabes.

Contexto:
{context}

Pregunta:
El usuario está preguntando sobre: {query}
"""

In [36]:
from openai import OpenAI

In [ ]:
#AQUI API KEY

In [38]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()


completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

print(completion.choices[0].message.content)

Las "stop words" o "stopwords" son palabras extremadamente comunes en un idioma que tienen poco valor a la hora de ayudar a seleccionar documentos que coincidan con la necesidad del usuario. Estas palabras suelen excluirse del vocabulario durante el proceso de indexación. Por ejemplo, términos como "y", "de" o "el" se consideran stop words y no se utilizan para evaluar la relevancia de los documentos porque aparecen con tanta frecuencia que no aportan información útil.

El uso de listas de stop words es una estrategia para reducir el tamaño de los índices en los sistemas de recuperación de información (IR), ya que permite descartar palabras que no contribuyen a la búsqueda. Sin embargo, el uso de stop words puede afectar a ciertos tipos de consultas, como las búsquedas en frases, donde incluso las palabras comunes pueden ser relevantes.
